# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.11.4.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [3]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter, PDFFileLoader
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [4]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using blogs from PMarca (Marc Andreessen) as our sample data. This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


#### Loading Text File

In [5]:
text_loader = TextFileLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

1

In [6]:
print(documents[0][:100])


The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horow


#### Loading PDFs
crack open the PDF and convert content to text

In [7]:
pdf_loader = PDFFileLoader("data/digitalskilljp.pdf")
pdf_documents = pdf_loader.load_documents()

### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [8]:
text_splitter = CharacterTextSplitter()
text_chunks = text_splitter.split_texts(documents)
len(text_chunks)

5959

For pdfs, there are many documents once cracked open. Need to use recursively split text. For simplicity, we're gonna use Langchain Library

In [9]:
pdf_chunks = text_splitter.split_texts(pdf_documents)
len(pdf_chunks)

275

In [10]:
# Combine all chunks
all_chunks = text_chunks + pdf_chunks

Let's take a look at some of the documents we've managed to split.

In [12]:
text_chunks[0:10]

['\ufeff\nThe Pmarca Blog Archives\n(select posts from 2007-2009)\nMarc Andreessen\ncopyright: Andreessen Horow',
 '-2009)\nMarc Andreessen\ncopyright: Andreessen Horowitz\ncover design: Jessica Hagy\nproduced using: Pre',
 'itz\ncover design: Jessica Hagy\nproduced using: Pressbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart',
 'ssbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart 1: Why not to do a startup 2\nPart 2: When the VCs',
 ' 1: Why not to do a startup 2\nPart 2: When the VCs say "no" 10\nPart 3: "But I don\'t know any VCs!" 1',
 ' say "no" 10\nPart 3: "But I don\'t know any VCs!" 18\nPart 4: The only thing that matters 25\nPart 5: T',
 '8\nPart 4: The only thing that matters 25\nPart 5: The Moby Dick theory of big companies 33\nPart 6: Ho',
 'he Moby Dick theory of big companies 33\nPart 6: How much funding is too little? Too much? 41\nPart 7:',
 "w much funding is too little? Too much? 41\nPart 7: Why a startup's initial business plan doesn't\nmat",
 " Why a startup's 

In [13]:
pdf_chunks[0:10]

['All Rights Reserved Copyright© IPA 2024 0＜概要編＞\nデジタルスキル標準\nver.1.2\n2024年7月\n\nAll Rights Reserved Copyri',
 'ルスキル標準\nver.1.2\n2024年7月\n\nAll Rights Reserved Copyright© IPA 2024 1デジタルスキル標準策定の背景・ねらい\n日本企業におけるDX 推進の重要',
 'ght© IPA 2024 1デジタルスキル標準策定の背景・ねらい\n日本企業におけるDX 推進の重要性の高まり\n•データ活用やデジタル技術の進化により、我が国や諸外国において、データ・デジタル技術を活',
 '性の高まり\n•データ活用やデジタル技術の進化により、我が国や諸外国において、データ・デジタル技術を活用した産業構造の変化が起きつつある。  このような変化の中\nで企業が競争上の優位性を確立するためには',
 '用した産業構造の変化が起きつつある。  このような変化の中\nで企業が競争上の優位性を確立するためには、常に変化する社会や顧客の課題を捉え、デジタルトランスフォーメーション（ DX脚注）を実現することが',
 '、常に変化する社会や顧客の課題を捉え、デジタルトランスフォーメーション（ DX脚注）を実現することが重要。\n•一方で、多くの日本企業は、  DXの取組みにおくれをとっていると考えられる。その大きな要因',
 '重要。\n•一方で、多くの日本企業は、  DXの取組みにおくれをとっていると考えられる。その大きな要因のひとつとして、 DXの素養や専門性を持った人材が不足してい\nることが挙げられる。\nDX推進における',
 'のひとつとして、 DXの素養や専門性を持った人材が不足してい\nることが挙げられる。\nDX推進における人材の重要性\n•企業が DXを実現するには、企業全体として変革への受容性を高める必要がある。そのため',
 '人材の重要性\n•企業が DXを実現するには、企業全体として変革への受容性を高める必要がある。そのためには、経営層を含め企業に所属する一人一人が DXの素養を\n持っている状態、すなわち DXに理解・関心',
 'には、経営層を含め企業に所属する一人一人が DXの素養を\

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [14]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
   - Yes, this particular embedding model can take up to 2048 dimensions or less. It can be changed inside of request body parameter `dimensions` . By default, the length of the embedding vector is 1536 for text-embedding-3-small.
2. What technique does OpenAI use to achieve this?
   1. Matryoshka Representation Learning. Matryoshka Representation Learning (MRL) is a novel approach in machine learning that aims to create flexible representations capable of adapting to various downstream tasks with different computational constraints. This method is inspired by the concept of Matryoshka dolls, also known as Russian nesting dolls, where each doll fits inside a larger one. Similarly, MRL encodes information at different granularities, allowing a single embedding to adapt to the computational constraints of downstream tasks This technique allows new embedding models like v3 to trade off performance and cost of using embedding (bigger model is more expensive) without losing the representation propeties. For example, on the MTEB benchmark, a text-embedding-3-large embedding can be shortened to a size of 256 while still outperforming an unshortened text-embedding-ada-002 embedding with a size of 1536.

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #2!

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [15]:
# Create chunks with source information whether text or pdf
text_chunks_with_source = [(chunk, "text") for chunk in text_chunks]
pdf_chunks_with_source = [(chunk, "pdf") for chunk in pdf_chunks]

# Combine all chunks with their sources
all_chunks_with_source = text_chunks_with_source + pdf_chunks_with_source

# Create vector database from combined chunks
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list([chunk for chunk, _ in all_chunks_with_source]))

Let see the comparison between distance metrics i.e. Cosine Similarity and Euclidean
Cosine Similarity scores will be between -1 and 1:
- Higher scores (closer to 1) indicate more similar texts
- The scores represent the angle between vectors
  
Euclidean Distance scores will be non-negative:
- Lower scores indicate more similar texts
- The scores represent the actual distance between vectors in the embedding space

In [20]:
# Compare both metrics
cosine_results, euclidean_results = vector_db.compare_distance_metrics("What does the Japan Government do to support digital skills?", k=2)

# Print results
print("Cosine Similarity Results:")
for text, score in cosine_results:
    print(f"Text: {text}")
    print(f"Score: {score:.4f}")
    print("---")

print("\nEuclidean Distance Results:")
for text, score in euclidean_results:
    print(f"Text: {text}")
    print(f"Score: {score:.4f}")
    print("---")

Cosine Similarity Results:
Text: 業省「デジタルガバナンス・コード 2.0」（2022年9 月改訂）  ）デジタルスキル標準の策定
•上記のような DX推進における人材の重要性を踏まえ、個人の学習や企業の人材確保・育成の指針であるデジ
Score: 0.5572
---
Text: ght© IPA 2024 1デジタルスキル標準策定の背景・ねらい
日本企業におけるDX 推進の重要性の高まり
•データ活用やデジタル技術の進化により、我が国や諸外国において、データ・デジタル技術を活
Score: 0.5449
---

Euclidean Distance Results:
Text: ce neither can be mine, let
me then tow to pieces, while still chasing thee, though tied to thee,
th
Score: 1.4815
---
Text: nded with such an
insupportable smell, as to bring on a disorder of the brain.”
— Ulloa’s South Amer
Score: 1.4629
---


Cosine Similarity Results:
- Both results are in Japanese and discuss digital governance and digital skills standards
- The scores are quite close (0.5572 and 0.5449)
- This makes sense as both texts are semantically related to digital governance and skills standards
- The high similarity scores indicate these texts are conceptually very close

Euclidean Distance Results:
- The results are in English and appear to be from a literary text
- The distances are quite different (1.4815 and 1.4629)
- These results seem unrelated to the cosine similarity results, which suggests there Euclidean doesn't perform well for this particular data and query characteristics

This comparison can help us understand which metric might be more suitable for our specific use case. Generally:
- Use cosine similarity when we want to compare the semantic similarity of texts
- Use Euclidean distance when you want to consider both direction and magnitude differences which depends on the user query. 

Here's a practical example: Let's say you're comparing product descriptions where the vector dimensions represent specific features with meaningful magnitudes (like price, size, weight, etc.).

products = [
    "Small laptop: 13-inch screen, 2.5kg weight, $999 price",
    "Medium laptop: 15-inch screen, 3.0kg weight, $1299 price",
    "Large laptop: 17-inch screen, 3.5kg weight, $1599 price",
    "Small tablet: 10-inch screen, 0.5kg weight, $499 price",
    "Large tablet: 12-inch screen, 0.7kg weight, $699 price"
]

query = "I want a lightweight device around 1kg with a medium-sized screen"


In this example, Euclidean distance might perform better because:

**Magnitude Matters:**
- The weight (0.5kg vs 3.5kg) is a meaningful difference
- Cosine similarity would only consider the direction of the vectors, ignoring that a 3.- 5kg difference is much more significant than a 0.2kg difference
  
**Physical Measurements:**
- Screen size (13-inch vs 17-inch) represents actual physical dimensions
- Price differences ($999 vs $1599) are meaningful magnitudes
- Euclidean distance captures these absolute differences better
  
**Specific Requirements:**
- When the query specifies exact requirements ("around 1kg"), Euclidean distance can better match these specific magnitude requirements
- Cosine similarity might match based on semantic similarity alone, ignoring the specific magnitude requirements


#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?
      - Synchronous APIs are advantageous, particularly for immediate response and real-time interaction. In the case of embedding, it allowds concurrent requests to be processed simultaneously. It also helps with latency, the total time it takes to retrieve all embeddings are noticably lower when we have to work with network latency. Scalability is probably the most important one as most applications need to handle a large volume of embedding requests, using async can improve scalablity by efficiently managing I/O-bound tasks without the overhead of creating many threads. Keep in mind that while using async can improve performance, you’ll also need to manage aspects like API rate limits and error handling carefully to ensure your application remains robust. Functionality between the synchronous and asynchronous clients is otherwise identical.

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [33]:
vector_db.search_by_text("AI and Digital Skills", k=5)

[('ights Reserved Copyright© IPA 2024 3\uf06cデジタルスキル標準で対象とする人材は、 デジタル技術を活用して競争力を向上させる企業等に所属する人材を想定している。\n\uf06cこのう',
  0.5111101392971779),
 ('戦略\nデータ・ AI活用業務の設計・事業実装・評価\nAI・データサイエンス 数理統計・多変量解析・データ可視化\n機械学習・深層学習\nデータエンジニアリング データ活用基盤設計データ活用基盤実装・運用\n',
  0.4948263495466843),
 ('用データ・ AIの\n戦略的活用データ理解・活用 b\nデータ・ AI活用戦略 c\nデータ・AI 活用業務の設計・事業実装・評価 b\nAI・\nデータサイエンス数理統計・多変量解析・データ可視化 a\n機械学',
  0.484380254156856),
 ('Reserved Copyright© IPA 2024 15デジタルスキル標準の活用・普及の取組について\n１ DXを推進する人材の育成に向けた教育コンテンツの充実\nデジタルスキル標準の普及に求めた方',
  0.4823655637975129),
 ('点）\nその他デザイン技術\nデータ活用データ・ AIの戦略的活用 データ理解・活用\nデータ・ AI活用戦略\nデータ・ AI活用業務の設計・事業実装・評価\nAI・データサイエンス 数理統計・多変量解析・デ',
  0.48201532812295733)]

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?
- Using RAG, grounding responses to data.
- Be detailed in and clear in the prompts to eliminate ambiguity
- Provide examples of models inputs and outputs aka few-shot learning
- Invest in creating evaluations (evals) for  prompts, using test data that looks like the data you expect to see in production. Due to the inherent variable results from different models, using evals to see how the prompts perform is the best way to ensure the prompts work as expected.

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

### Creating and Prompting OpenAI's `gpt-4o-mini`!

Let's tie all these together and use it to prompt `gpt-4o-mini`!

In [34]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="AI"),
    user_role_prompt.create_message(
        content="What does the Japan Government do to support digital skills?"
    ),
]

response = chat_openai.run(messages)

In [35]:
print(response)

The Japanese government has implemented various initiatives to support and enhance digital skills across the country. Here are some key efforts:

1. **National Strategy for Digital Transformation**: Japan has developed a national strategy aimed at promoting digital transformation in various sectors. This includes fostering digital literacy and skills among the workforce.

2. **Education Reforms**: The government has initiated reforms in educational curricula to incorporate digital literacy from an early age. This includes the introduction of programming and information technology education in elementary and secondary schools.

3. **Public-Private Partnerships**: Collaboration with private companies and tech organizations helps provide training programs and workshops focused on enhancing digital skills. These partnerships aim to ensure that individuals are equipped with the necessary skills for a rapidly changing job market.

4. **Support for Workforce Development**: The government prov

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [39]:
RAG_PROMPT_TEMPLATE = """ \
Use the provided context to answer the user's query.

You may not answer the user's query unless there is specific context in the following text.

If it's another language, always translate it to English.

If you do not know the answer, or cannot answer, please respond with "I don't know".
"""

rag_prompt = SystemRolePrompt(RAG_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """ \
Context:
{context}

User Query:
{user_query}
"""


user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)

        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = rag_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)

        return {"response" : self.llm.run([formatted_system_prompt, formatted_user_prompt]), "context" : context_list}

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?
- We can encourage the LLM to handle more complex tasks throughs step by step thinking

What is that strategy called?
- Chain of Thought

> NOTE: You can look through ["Accessing GPT-3.5-turbo Like a Developer"](https://colab.research.google.com/drive/1mOzbgf4a2SP5qQj33ZxTz2a01-5eXqk2?usp=sharing) for an answer to this question if you get stuck!

In [40]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)

In [41]:
retrieval_augmented_qa_pipeline.run_pipeline("What is the Japan Government doing to support digital skills?")

{'response': 'The Japan Government is formulating a "Digital Skills Standard" to support the cultivation of talent necessary for promoting digital transformation (DX). This initiative recognizes the importance of digital skills in companies and aims to provide guidance for individual learning and talent acquisition and development within organizations. Additionally, it involves enhancing educational content aimed at promoting the skill development of personnel involved in digital transformation.',
 'context': [('業省「デジタルガバナンス・コード 2.0」（2022年9 月改訂）  ）デジタルスキル標準の策定\n•上記のような DX推進における人材の重要性を踏まえ、個人の学習や企業の人材確保・育成の指針であるデジ',
   0.5604267988416086),
  ('ght© IPA 2024 1デジタルスキル標準策定の背景・ねらい\n日本企業におけるDX 推進の重要性の高まり\n•データ活用やデジタル技術の進化により、我が国や諸外国において、データ・デジタル技術を活',
   0.5449089788786071),
  ('省が主催する「デジタル時代の人材政策に関する検討会」において、「デジタルスキル標準（DSS) 」として取りまとめ\nました。※所属は2022年12 月21日現在のものです\n',
   0.541160585728317),
  ('Reserved Copyright© IPA 2024 15デジタルスキル標準の活用・普及の取組について\n１ DXを推進する人材の育成に向けた教育コンテンツの充実\nデジタルスキル標準の普及に求め

### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database

While these are suggestions, you should feel free to make whatever augmentations you desire! 

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

In [ ]:
### YOUR CODE HERE